In [18]:
# Imports
from ncps.wirings import AutoNCP
from ncps.torch import CfC
import pytorch_lightning as pl
from pytorch_lightning.loggers import CSVLogger
from numpy import genfromtxt
import numpy as np
import torch
import torch.utils.data as data
import matplotlib as plt
import torch.nn as nn
from random import sample
import os

torch.set_float32_matmul_precision("high")

batchSize = 64

In [2]:
# FORMATTING DATASET FOR FED. LEARNING
dataFile = 'Data/CfCMultiExtension/RandomPos_0709.csv'
dataSet = genfromtxt(dataFile, delimiter=',')
dataSet = np.delete(dataSet, 0, axis=0) # Remove the labels at the beginning of the dataset

# Devide dataset into reciever groups
fedDataSet = dataSet[np.argsort(dataSet[:, 1])] # Sort dataset by reciver ID
_, counts = np.unique(fedDataSet[:,1], return_counts=True) # Get the indexes of the change in datasets.
sum = 0
for i in range(len(counts)): # Accumulating counts so that we can use them as indexes
    sum += counts[i]
    counts[i] = sum
fedDataSet = np.split(fedDataSet, counts) # Split larger dataset into per vehicle datasets.

newData = [] 
for reciever in fedDataSet: # Go through all vehicle datasets
    subData = []
    index = 0
    while index < len(reciever) - 10: # organize the new dataset as a list of chuncks of 10 messages 
        subData.append(reciever[index:index+10])
        index += 5
    subData = torch.Tensor(subData)
    if subData.shape[0] != 0:
        newData.append(subData) # Create tensor from per vehicle dataset and add to list of datas.
fedDataSet = newData
# Final output of this cell is fedDataSet, a list of the datasets of each vehicle.

/var/folders/6k/3_3skq9j04g6tdh6317v_l200000gn/T/ipykernel_68290/2495743071.py:22: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:257.)
  subData = torch.Tensor(subData)


In [3]:
print(fedDataSet[25].shape)

torch.Size([56, 10, 12])


In [4]:
# ***** DEPRECATED *****
'''
# #Current Simulation File
dataFile = 'Data/CfCMultiExtension/RandomPos_0709.csv'

dataSet = genfromtxt(dataFile, delimiter=',')

batchSize = 64
# Ceate dataloader and fill with (BSM, attk#). Expanding to add 0th dimension for batches.
# Batch size should be 64 for the low density simulations and 128 for high density simulations.
# No shuffle to keep batches on same vehicle.
# Num_workers is set to = num CPU cores
dataSet[0:-1,:] = dataSet[1:,:] # Get rid of the first null value of the dataset
# Sort Dataset by reciever id to pass on to every car.
dataSet = dataSet[np.argsort(dataSet[:, 1])]
print(dataSet.shape)
# count subsets per vehicle
print(dataSet[0])
unq, counts = np.unique(dataSet[:, 1], return_counts = True)
recvr = 0
lastRecieverCount = 0
newData = []
print(counts)
# Organize dataset into sets of 10 messages by reciever
while recvr < counts.shape[0]:
    # Loop through reciever
    index = 0
    while index < counts[recvr] - 10:
        # Loop through messages from reciever
        newData.append(dataSet[lastRecieverCount+index:lastRecieverCount +index+10])
        index += 5
    recvr += 1
    lastRecieverCount = counts[recvr-1]
dataSet = torch.tensor(newData)
print(dataSet.shape)
'''

"\n# #Current Simulation File\ndataFile = 'Data/CfCMultiExtension/RandomPos_0709.csv'\n\ndataSet = genfromtxt(dataFile, delimiter=',')\n\nbatchSize = 64\n# Ceate dataloader and fill with (BSM, attk#). Expanding to add 0th dimension for batches.\n# Batch size should be 64 for the low density simulations and 128 for high density simulations.\n# No shuffle to keep batches on same vehicle.\n# Num_workers is set to = num CPU cores\ndataSet[0:-1,:] = dataSet[1:,:] # Get rid of the first null value of the dataset\n# Sort Dataset by reciever id to pass on to every car.\ndataSet = dataSet[np.argsort(dataSet[:, 1])]\nprint(dataSet.shape)\n# count subsets per vehicle\nprint(dataSet[0])\nunq, counts = np.unique(dataSet[:, 1], return_counts = True)\nrecvr = 0\nlastRecieverCount = 0\nnewData = []\nprint(counts)\n# Organize dataset into sets of 10 messages by reciever\nwhile recvr < counts.shape[0]:\n    # Loop through reciever\n    index = 0\n    while index < counts[recvr] - 10:\n        # Loop thr

In [5]:
#PROPER FORMATTING FOR TESTING DATASETS
#Time sequences are 10 timepoints (Messages) with 7 features per message.
#Organized by car.
unq, counts = np.unique(dataSet[:, 2], return_counts = True)
sender = 0
lastSenderCount = 0
newData = []
# Organize dataset into sets of 10 messages by sender
while sender < counts.shape[0]:
    # Loop through sender
    index = 0
    while index < counts[sender] - 10:
        # Loop through messages from sender
        newData.append(dataSet[lastSenderCount+index:lastSenderCount +index+10])
        index += 5
    sender += 1
    lastSenderCount += counts[sender-1]
dataSet = torch.tensor(newData)
leng = dataSet.shape[0]
trainPerc = 80
# Create seperate datasets for testing and training, using Train Percentage as metric for split
trainDataIn = torch.Tensor(dataSet[:int(leng*(trainPerc/100)),:,3:10]).float()
trainDataOut = torch.Tensor(np.int_(dataSet[:int(leng*(trainPerc/100)),:,11])).long()
testDataIn = torch.Tensor(dataSet[int(leng*(trainPerc/100)):,:,3:10]).float()
testDataOut = torch.Tensor(np.int_(dataSet[int(leng*(trainPerc/100)):,:,11])).long()
newsetIn = []
newsetOut = []
testsetIn = []
testsetOut = []
# Create dataset of 1/100th of the entries for quicker testing during development
for index in range(0,int((leng) * (trainPerc/100))):
    if not (int(index/10) % 100):
        newsetIn.append(dataSet[index,:,3:10])
        newsetOut.append((dataSet[index,:,11]))
for idx in range(int((leng) * (trainPerc/100)), leng):
    if not (int(idx/10) % 10):
        testsetIn.append(dataSet[idx,:,3:10])
        testsetOut.append((dataSet[idx,:,11]))
testingIn = torch.Tensor(np.array(newsetIn)).float()
testingOut = torch.Tensor(np.array(newsetOut)).long()
inTest = torch.Tensor(np.array(testsetIn)).float()
outTest = torch.Tensor(np.array(testsetOut)).long()
# Create Dataloaders for all the datasets
dataLoaderTrain = data.DataLoader(data.TensorDataset(trainDataIn, trainDataOut), batch_size=batchSize, shuffle=False, num_workers=10, persistent_workers = True, drop_last= True)
dataLoaderTest = data.DataLoader(data.TensorDataset(testDataIn, testDataOut), batch_size=batchSize, shuffle=False, num_workers=10, persistent_workers = True, drop_last= True)
testingDataLoader = data.DataLoader(data.TensorDataset(testingIn, testingOut), batch_size=batchSize, shuffle = False, num_workers=10, persistent_workers = True, drop_last= True)
testingTestData = data.DataLoader(data.TensorDataset(testingIn, testingOut), batch_size=batchSize, shuffle = False, num_workers=10, persistent_workers = True, drop_last= True)


In [6]:
# TESTS
print(inTest.shape)
print(outTest.shape)
print()

torch.Size([12480, 10, 7])
torch.Size([12480, 10])



In [7]:
#TESTING
tom = np.array([[1,2,3,4],[9,8,7,6],[5,7,8,3],[9,8,7,3]])
print(tom)
tom = tom[np.argsort(tom[:, 1])]
print(tom)

[[1 2 3 4]
 [9 8 7 6]
 [5 7 8 3]
 [9 8 7 3]]
[[1 2 3 4]
 [5 7 8 3]
 [9 8 7 6]
 [9 8 7 3]]


In [8]:
# Creating Learner
class CfCLearner(pl.LightningModule):
    def __init__(self, model, lr):
        super().__init__()
        self.model = model
        self.lr = lr
        self.lossFunc = nn.CrossEntropyLoss()
        self.loss = None
    
    def training_step(self, batch, batch_idx):
        # Get in and out from batch
        inputs, target = batch
        # Put input through model
        output, _ = self.model.forward(inputs)
        # Reorganize inputs for use with loss function
        output = output.permute(0, 2, 1)
        # Calculate Loss using Cross Entropy Loss 
        loss = self.lossFunc(output, target)
        self.log("trainLoss", loss, prog_bar=True)
        self.loss = loss
        return loss

    def validation_step(self, batch, batch_idx):
        # Get in and out from batch
        inputs, target = batch
        # Put input through model
        output, _ = self.model.forward(inputs)
        # Reorganize inputs for use with loss function
        output = output.permute(0, 2, 1)
        print(f"output: {output.shape}")
        print(f"target: {target.shape}")
        # Calculate Loss using Cross Entropy Loss 
        loss = self.lossFunc(output, target)
        self.log("valLoss", loss, prog_bar=True)
        self.loss = loss
        return loss

    def test_step(self, batch, batch_idx):
        return self.validation_step(batch, batch_idx)

    def configure_optimizers(self):
        # Using AdamW optomizer based on info from paper
        # optimizer = torch.optim.AdamW(self.model.parameters(), lr = 0.001)
        # return ([optimizer], [torch.optim.lr_scheduler.ExponentialLR(optimizer, 0.6)])
        return torch.optim.AdamW(self.model.parameters(), lr = 0.01) # TESTING REMOVING THE SCHEDULER

In [9]:
# Creating Model/Module
class Modena(nn.Module): 
    # CfC with feed-forward layer to classify at end.
    def __init__(self, inputSize, unitNum = None, motorNum = 2, outputDim = 2, batchFirst = True):
        super().__init__()
        if isinstance(inputSize, Modena):
            self.inputSize = inputSize.inputSize
            self.unitNum = inputSize.unitNum
            self.motorNum = inputSize.motorNum
            self.outputDim = inputSize.outputDim
            self.batchFirst = inputSize.batchFirst
            # Create NCP wiring for CfC
            wiring = AutoNCP(self.unitNum, self.motorNum)
            # Create CfC model with inputs and wiring
            self.cfc = CfC(self.inputSize, wiring, batch_first=self.batchFirst)
            # Create feed-forward layer
            self.fF = nn.Linear(self.motorNum, self.outputDim)
            self.fF.weight = nn.Parameter(inputSize.fF.weight)
        else:
            self.inputSize = inputSize
            self.unitNum = unitNum
            self.motorNum = motorNum
            self.outputDim = outputDim
            self.batchFirst = batchFirst
            # Create NCP wiring for CfC
            wiring = AutoNCP(unitNum, motorNum)
            # Create CfC model with inputs and wiring
            self.cfc = CfC(inputSize, wiring, batch_first=batchFirst)
            # Create feed-forward layer
            self.fF = nn.Linear(motorNum, outputDim)
        

    def forward(self, batch, hidden = None):
        batch, hidden = self.cfc(batch, hidden) # Pass inputs through CfC
        out = nn.functional.relu(self.fF(batch)) # pass through FeedForward Layer, then make 0 minimum
        return out, hidden # Return the guess and the hidden state

In [14]:
# Creating overall model Class
class OBU():
    def __init__(self, inputSize, units = 20, motors = 8, outputs = 20, epochs = 10, lr = 0.01, randInt = 0, gpu = False, dataset = None, evil = False):
        if isinstance(inputSize, OBU):
            self.lr = inputSize.lr
            self.epochs = inputSize.epochs
            self.gpu = inputSize.gpu
            self.model = Modena(inputSize.model)
            self.model.load_state_dict(inputSize.model.state_dict())
            self.learner = CfCLearner(self.model, self.lr)
            self.learner.load_state_dict(inputSize.learner.state_dict())
            self.trainer = pl.Trainer(
                logger = CSVLogger('log/Fed'), # Set ouput destination of logs, logging accuracy every 50 steps
                max_epochs = self.epochs, # Number of epochs to train for
                gradient_clip_val = 1, # This is said to stabilize training, but we should test if that is true
                accelerator = "gpu" if self.gpu else "cpu" # Using the GPU to run training or not
                )

        else:
            self.lr = lr
            self.epochs = epochs
            self.gpu = gpu
            self.model = Modena(inputSize, units, motors, outputs)
            self.learner = CfCLearner(self.model, lr) # tune units, lr
            self.trainer = pl.Trainer(
                logger = CSVLogger('log/Fed'), # Set ouput destination of logs, logging accuracy every 50 steps
                max_epochs = epochs, # Number of epochs to train for
                gradient_clip_val = 1, # This is said to stabilize training, but we should test if that is true
                accelerator = "gpu" if gpu else "cpu" # Using the GPU to run training or not
                )
        self.prevAccuracy = 0
        self.evil = evil
        self.nearbyOBUs = []
        self.id = None
        self.dataset = dataset
        self.outnum = 0
        self.confidences = []
        self.samplingWeights = []
        self.priority = 0
        self.datalen = 0
        self.otherPriorities = []
        self.sampling = []
        self.curr_loss = 0
        self.prev_loss = None
        self.trust_loss = 0
        self.backupWeights = {'model':self.model.state_dict(), 'learner':self.learner.state_dict()}
        self.prevWeights = dict(self.model.state_dict())
    

        # Overloading add function to create fed.avg. model
    def __add__(self, other):
        # if self.learner.getHidden() != None and other.learner.getHidden() != None:
        self.model.load_state_dict(dict( (n, self.model.state_dict().get(n, 0)+other.model.state_dict().get(n, 0)) for n in set(self.model.state_dict())|set(other.model.state_dict()) ))
        # elif other.learner.getHidden() != None:
        #     self.model.load_state_dict(other.model.state_dict())
        # elif self.learner.getHidden() != None:
        #     self.model.load_state_dict(self.model.state_dict())
        return self
    
    def __mul__(self, i):
        self.model.load_state_dict(dict((n, self.model.state_dict().get(n, 0)*i) for n in self.model.state_dict()))
        return self

    # Overloading div. function to average model
    def __truediv__(self, i):
        
        self.model.load_state_dict(dict((n, self.model.state_dict().get(n, 0)/i) for n in self.model.state_dict()))
        # self.model.load_state_dict(self.model.state_dict()/i)
        # self.learner.setHidden(self.learner.getHidden() / i)
        # self.model.fF.weight = nn.Parameter(self.model.fF.weight/i)
        return self
    
    def fit(self, dataLoader):
        # calling built in fit function
        self.trainer.fit(self.learner, dataLoader) 
        return self.learner.loss
    
    # Function to run model through a testing dataset and calculate accuracy. Can be expanded to give more metrics and more useful metrics.
    def test(self, dataIn, dataOut):
        # Put input data through model and determine classification
        with torch.no_grad():
            outs = np.asarray(self.model(dataIn)[0])
        outs = torch.from_numpy(outs)
        # Get the label with the maximum confidence for determining classification
        print(outs.shape)
        _, res = torch.max(outs, 2)
        Pt = Pf = Nt = Nf = 0
        countR = 0
        numZero = 0
        tot = outs.shape[0]
        total = 0
        for i in range(0, tot):
            # Loop through sequences of 10 each
            for t in range(0, res[i].shape[0]):
                # Loop through the sub-sequences
                if res[i,t] == dataOut[i,t]:
                    if res[i,t] == 0:
                        Nt += 1
                        numZero += 1
                    else:
                        Pt += 1
                    # Check if label is correct, and add to count right accordingly
                    countR += 1
                else:
                    if dataOut[i,t] == 0:
                        Pf += 1
                        numZero += 1
                    else:
                        Nf += 1
                total += 1
        # Calculate percent correct and percent zero
        if Pt != 0:
            accuracy = (Pt+Nt)/(Pt+Pf+Nf+Nt)
            precision = (Pt)/(Pt+Pf)
            recall = (Pt)/(Pt+Nf)
            f1 = (2*precision*recall)/(precision+recall)
            print(precision)
            print(recall)
            print("Model got " + str(countR) + "/" + str(total) + " right.")
            print(f"Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}, F1 Score: {f1}")
            print(f"{numZero}, {numZero/total * 100}% Zeroes, {total-numZero} Non Zero entries.")
            return f"Model got {countR}/{total} right. Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}, F1 Score: {f1}"
        else:
            print("Model could not complete tests.")
            return f"Model could not complete tests, found 0 of misbehaviour."
    
    def testStep(self, dataLoader):
        self.learner.validation_step(next(iter(dataLoader)), 0)
    
    def setModel(self, model):
        if not model == None:
            self.model = model

    def getModel(self):
        return self.model
    
    def getSavedState(self):
        return self.prevWeights
    
    def updateSavedStates(self):
        if self.evil:
            return # Never update weights, so always passing on Zero weights. Can also try with infite/random weights
        self.prevWeights = self.model.state_dict()
    
    def getState(self):
        return self.model.state_dict()
    
    def restoreFromBackup(self):
        self.model.load_state_dict(self.backupWeights['model'])
        self.learner.load_state_dict(self.backupWeights['learner'])

    def saveBackup(self):
        self.backupWeights['model'] = self.model.state_dict()
        self.backupWeights['learner'] = self.learner.state_dict()
    
    def isEvil(self):
        return True if self.evil else False
    
    def setState(self, one, two = None):
        if two:
            tom = dict((n, one.get(n, 0)+two.get(n, 0)) for n in set(one)|set(two))
        else:
            tom = one
        self.model.load_state_dict(tom)
        return tom
    
    def step(self, epochs):
        self.trainer.fit_loop.max_epochs = self.trainer.current_epoch + epochs
        self.curr_loss = self.fit(self.dataset)
        return self.curr_loss

    def updateSelected(self):
        # self.sampling = [self.nearbyOBUs[i] for i in torch.utils.data.WeightedRandomSampler([self.samplingWeights[i] for i in self.nearbyOBUs], self.outnum)] # Randomly generates list of outnum vehicles to sample based on the sampling weights.
        # print(self.sampling)
        # return self.sampling
        self.sampling = []
        count = 0
        for idx in self.nearbyOBUs:
            rand = np.random.randint(0, 100)
            if rand <= int(100*self.samplingWeights[idx]): # Less than or equal, as we want 1 to be selected every time.
                self.sampling.append(int(idx))
                count += 1
        return self.sampling # Returning how many vehicles were selected for training
        
        
    def resetTrainer(self):
        self.trainer = pl.Trainer(
            logger = CSVLogger('log'), # Set ouput destination of logs, logging accuracy every 50 steps
            max_epochs = self.epochs, # Number of epochs to train for
            gradient_clip_val = 1, # This is said to stabilize training, but we should test if that is true
            accelerator = "gpu" if self.gpu else "cpu" # Using the GPU to run training or not
            )


In [ ]:
# Standard Federated Learning

# With 2 epochs, 10 sub, and 0:3 models ending acc. of 0.1377%
# With 2 epochs, 10 sub, and 0:10 models ending acc. of

# Tested with weights, but weighing off of the loss leads to choosing the models trained by vehicles without attacks.

pl.seed_everything(1000)
results = {}
dataSets = {}
models = {}
histWeights = []
percentages = []
cars = []
rcvrs = []
percs = {}
states = {}
subEpochs = 50 # 50
epochs = 5 # 5
vehicles = 50 # 50
lr = 0.01
motors = 8
units = 20
batchSize = 64
gpu = False
deepTest = False
weighing = False
randomVehicles = False
doValidation = False
# Create starting models
mainModel = OBU(7, epochs= subEpochs, gpu = gpu, lr = lr, motors = motors, units = units)
nextModel = OBU(7, epochs= subEpochs, gpu = gpu, lr = lr, motors = motors, units = units)

if not randomVehicles:
    # Divide dataset of recieving vehicles among OBUs
    rcvrs = []
    for vehicle in fedDataSet[500:vehicles+500]: # 10
        rcvrID = int(vehicle[0,0,2].item())
        # Add new OBU for each model
        models[rcvrID] = OBU(7, epochs = subEpochs, gpu=gpu, lr = lr, motors = motors, units = units)
        # Create Slice of dataset
        vehicle = data.DataLoader(data.TensorDataset(vehicle[:,:,3:10].float(), vehicle[:,:,11].long()), batch_size=batchSize, shuffle=False, num_workers=16, persistent_workers = True) # type: ignore
        # Add sub - dataset to dataset
        rcvrs.append(rcvrID)
        dataSets[rcvrID]=vehicle
        models[rcvrID].dataset = vehicle

# Train individual models and combine
for epoch in range(epochs):
    i = 0
    if randomVehicles:
        # choose random assortment of vehicles to train on
        cars = []
        rcvrs = []
        for ted in np.random.choice(len(fedDataSet), vehicles, replace = False):
            cars.append(fedDataSet[ted])
        # Divide dataset of recieving vehicles among OBUs
        for vehicle in cars: # 10
            rcvrID = int(vehicle[0,0,2].item())
            # Add new OBU for each model
            if rcvrID not in models:
                models[rcvrID] = OBU(7, epochs = subEpochs, gpu=gpu, lr = lr, motors = motors, units = units)
            # Create Slice of dataset
            vehicle = data.DataLoader(data.TensorDataset(vehicle[:,:,3:10].float(), vehicle[:,:,11].long()), batch_size=batchSize, shuffle=False, num_workers=16, persistent_workers = True)
            # Add sub - dataset to dataset
            rcvrs.append(rcvrID)
            dataSets[rcvrID]=vehicle
            models[rcvrID].dataset = vehicle

    print(rcvrs, file = open('rcvrs.txt', 'w'))
    # Baseline model to add everything to. !!Do I want this or should it be a completely new model?!! Got 0% on combination before, testing with new model for next model.
    nextModel = OBU(7, epochs= subEpochs, gpu = gpu, lr = lr, motors = motors, units = units)
    # Train models
    weights = []
    for rcvr in rcvrs:
        # Make multithreaded?
        if doValidation and models[rcvr].prevAccuracy != 0:
            _, _, p, _ = mainModel.test(inTest, outTest)
            print(f"Current Epoch: {epoch}, Reciever: {rcvr}")
            print(f"Tested. main perc: {p}, my perc: {models[rcvr].prevAccuracy}")
            if p > models[rcvr].prevAccuracy:
                print("Updating Model")
                # set model to main model, and train that 
                models[rcvr].setState(mainModel.getState())
        else:
            models[rcvr].setState(mainModel.getState())
        # Reset the trainer (should be unneeded now) to allow for further training
        # mod.resetTrainer()
        # Actually train
        modLoss = models[rcvr].step(subEpochs)
        weights.append(1/modLoss.item())
        if deepTest or doValidation:
            _, _ , perc, _ = models[rcvr].test(inTest, outTest)
            percs[rcvr] = perc
            models[rcvr].prevAccuracy = perc
            # Test individual model
            print(rcvr)
            if rcvr not in results:
                results[rcvr] = ([epoch, i+1, perc, modLoss.item()])
            else:
                results[rcvr].append([epoch, i+1, perc, modLoss.item()])
        states[rcvr] = (models[rcvr].getState())
        i+=1
        
    # Create combined model
    # combine models
    weights = np.abs(weights)/np.sum(weights)

    y=0
    for w in weights:
        y+= w
    teds = [[float(weights[n]), percs[n]] for n in range(len(percs))]
    percentages.append(teds)
    histWeights.append([weights,y])
    tom = 1/i
    for rcvr in rcvrs:
        if weighing:
            nextState = nextModel.setState(nextModel.getState(), dict((n, states[rcvr].get(n, 0)*weights[rcvr]) for n in states[rcvr])) # Done with weights
        else:
            nextState = nextModel.setState(nextModel.getState(), states[rcvr]) # No Weights
    if weighing:
        mainModel.setState(nextState) #Weights
    else:
        mainModel.setState(dict((n, nextState.get(n, 0)/i) for n in nextState)) # No weights
# Test combined model at end
perc = mainModel.test(testDataIn, testDataOut)
results['FINAL'] = [-1, -1, perc]
print(results, file = open('out/SF/results.txt', 'w'))
print(histWeights, file = open('out/SF/Weights.txt', 'w'))
print(percentages, file = open('out/SF/Percs.txt', 'w'))


# 'Model got 340703/1247740 right. Accuracy: 0.27305608540240756, Precision: 0.27978235144854047, Recall: 0.919080118694362, F1 Score: 0.42897730670851897'
# This was with scheduler, 50, 10, 100. Before accuracy with this was 96.388%, now 27.306%. 
# Running test with 50, 5, 50, w/out scheduler: 'Model got 1170620/1247740 right. Accuracy: 0.9381922515908763, Precision: 0.8443495151097161, Recall: 0.9709495548961424, F1 Score: 0.90323495386345'
# Accuracy of 93.820% after half the vehicles and half the epochs. I think we need to rething the scheduler.

Seed set to 1000
Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/opt/anaconda3/envs/Kettering/lib/python3.13/site-packages/pytorch_lightning/trainer/setup.py:177: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.
Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
HPU available: F

Epoch 49: 100%|██████████| 2/2 [00:00<00:00, 108.48it/s, v_num=2562, trainLoss=2.780]

`Trainer.fit` stopped: `max_epochs=50` reached.


Epoch 49: 100%|██████████| 2/2 [00:00<00:00, 90.44it/s, v_num=2562, trainLoss=2.780] 



  | Name     | Type             | Params | Mode 
------------------------------------------------------
0 | model    | Modena           | 1.6 K  | train
1 | lossFunc | CrossEntropyLoss | 0      | train
------------------------------------------------------
1.3 K     Trainable params
264       Non-trainable params
1.6 K     Total params
0.006     Total estimated model params size (MB)
27        Modules in train mode
0         Modules in eval mode
/opt/anaconda3/envs/Kettering/lib/python3.13/site-packages/pytorch_lightning/loops/fit_loop.py:310: The number of training batches (1) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


...

Epoch 249: 100%|██████████| 2/2 [00:00<00:00, 80.19it/s, v_num=2603, trainLoss=1.820]



  | Name     | Type             | Params | Mode 
------------------------------------------------------
0 | model    | Modena           | 1.6 K  | train
1 | lossFunc | CrossEntropyLoss | 0      | train
------------------------------------------------------
1.3 K     Trainable params
264       Non-trainable params
1.6 K     Total params
0.006     Total estimated model params size (MB)
27        Modules in train mode
0         Modules in eval mode


Epoch 249: 100%|██████████| 1/1 [00:00<00:00, 85.44it/s, v_num=2604, trainLoss=2.000] 

`Trainer.fit` stopped: `max_epochs=250` reached.


Epoch 249: 100%|██████████| 1/1 [00:00<00:00, 64.89it/s, v_num=2604, trainLoss=2.000]
torch.Size([124774, 10, 20])
0.8443495151097161
0.9709495548961424
Model got 1170620/1247740 right.
Accuracy: 0.9381922515908763, Precision: 0.8443495151097161, Recall: 0.9709495548961424, F1 Score: 0.90323495386345
877040, 70.29028483498165% Zeroes, 370700 Non Zero entries.


In [19]:
# DeFTA: Decentralized Federalized Training

pl.seed_everything(1000)

vehicleNumTot = 50 # 50
subNetworkNum = 15 # 15
totEpochs = 30 # 5
stepsPerEpoch = 5 # 30
minConnnectedVehicles = 10 # 10
backupThreshold = 3
vehicles = []
selectionWeights = {}
gpu = False
doEvil = True
percEvil = 10
lr = 0.01

path = f"DeFTA/{vehicleNumTot}-{subNetworkNum}-{totEpochs}-{stepsPerEpoch}-{minConnnectedVehicles}-{backupThreshold}/"
if not os.path.exists(f"out/{path}"):
    os.makedirs(f"out/{path}")
# Function to update sampling weights and confidence values
def phi(vehicle):
    m = [0 for n in range(vehicleNumTot)]
    for t in vehicle.sampling:
        m[t] += 1  # define matrix m that contains weather the vehicle is in the sampled set, and how many times it is in the set.
    if vehicle.prev_loss != None: # If it isn't the first iteration:
        if vehicle.curr_loss > vehicle.prev_loss * backupThreshold: # If this training round broke the model 
            print(f"Previous Loss: {vehicle.prev_loss}, Current loss: {vehicle.curr_loss}")
            print("Loading From Backup")
            vehicle.restoreFromBackup() # Restore backup weights
            vehicle.step(stepsPerEpoch) # Training Step
            vehicle.trust_loss = 10000 # set loss to infinity, as this model is destroyed
        else:
            if vehicle.curr_loss < vehicle.prev_loss: # If this is new best model
                vehicle.saveBackup() # Save model as new backup
            vehicle.trust_loss = vehicle.curr_loss - vehicle.prev_loss # Set change in trust
            vehicle.prev_loss = vehicle.curr_loss # Update previous loss to stay current
        print(vehicle.confidences)
        for i in range(len(vehicle.confidences)):
            vehicle.confidences[i] = vehicle.confidences[i] - (m[i]*vehicle.otherPriorities[i]*vehicle.trust_loss) # Update confidences based on what vehicles are contributing to the training and the size of their contribution
        for i in range(len(vehicle.confidences)):
            if vehicle.confidences[i] > 5:
                vehicle.confidences[i] = 5 # Add max to the sampling weights
        for i in range(len(vehicle.samplingWeights)):
            vehicle.samplingWeights[i] = (0.2 * vehicle.confidences[i]) if (vehicle.confidences[i] > 0) else (vehicle.confidences[i]) # Perform cRELU on C, with weighting a as 0.2 as per the findings of the DeFTA paper
        # vehicle.samplingWeights = np.exp(vehicle.samplingWeights)/np.sum(np.exp(vehicle.samplingWeights)) # Implementation of the softMax function to align the theta values properly. Trying without this to have everything sampled unless issue.
        for i in range(len(vehicle.samplingWeights)):
            if vehicle.samplingWeights[i] > 1:
                vehicle.samplingWeights[i] = 1 # Add max to the sampling weights
    else:
        vehicle.prev_loss = vehicle.curr_loss # Update previous loss while allowing for one epoch of randomness.
    if vehicle.id in selectionWeights:
        selectionWeights[vehicle.id].append(vehicle.samplingWeights)
    else:
        selectionWeights[vehicle.id] = [vehicle.samplingWeights]
    # Returns nothing, since all operations are done inside the vehicle

def updatePriorities(vehicles):
    for i in range(vehicleNumTot):
        vehicles[i].outnum = len(vehicles[i].sampling) + 1# Update useful outnumber of each vehicle in the simulation by having outnum = num sampled vehicles

    for i in range(vehicleNumTot): # Loop through vehicles and add priority of vehicle, done in separete loop as it requires info from other vehicles
        sum = vehicles[i].datalen/vehicles[i].outnum # Start out by including this vehicle's priority
        dum = vehicles[i].datalen/vehicles[i].outnum
        for j in vehicles[i].sampling:
            sum += vehicles[j].datalen/vehicles[j].outnum
        vehicles[i].priority = (vehicles[i].datalen/vehicles[i].outnum)/(sum)
        for j in vehicles[i].sampling:
            vehicles[i].otherPriorities[j] = (vehicles[j].datalen/vehicles[j].outnum)/sum # fill out standard list of other priorities
            dum += vehicles[j].datalen/vehicles[j].outnum
        print(f"vehicle {i} priority: {vehicles[i].priority}, total priority of subgroup: {dum/sum}")

for i in range(vehicleNumTot):
    set = data.DataLoader(data.TensorDataset(fedDataSet[i][:,:,3:10].float(), fedDataSet[i][:,:,11].long()), batch_size=batchSize, shuffle=False, num_workers=10, persistent_workers = True) # Create datasets
    if doEvil:
        if np.random.randint(0,100) < percEvil:
            vehicles.append(OBU(inputSize=7, units=20, motors=8, outputs=20, lr=lr, randInt=i, gpu=gpu, dataset=set, evil=True)) # Create evil vehicles
        else:
            vehicles.append(OBU(inputSize=7, units=20, motors=8, outputs=20, lr=lr, randInt=i, gpu=gpu, dataset=set)) # Create vehicles
    else:
        vehicles.append(OBU(inputSize=7, units=20, motors=8, outputs=20, lr=lr, randInt=i, gpu=gpu, dataset=set)) # Create vehicles
    vehicles[i].prevWeights = vehicles[i].getState() # Save previous state, so that we can do it in iterations
    vehicles[i].datalen = fedDataSet[i].shape[0]
    vehicles[i].outnum = np.random.randint(minConnnectedVehicles, subNetworkNum) # Get number of vehicles in sub network, at least #x so that vehicle has some use.
    range1 = np.arange(0, i).tolist()
    range2 = np.arange(i+1, vehicleNumTot).tolist()
    vehicles[i].nearbyOBUs = np.random.choice(range1 + range2, vehicles[i].outnum, replace=False) # Create subnetworks and add them to the vehicle
    vehicles[i].confidences = np.full((vehicleNumTot), 4.) # Initialize confidence values for all vehicles - can be shifted to a dict later to allow for varying number/discovery of vehicles
    vehicles[i].samplingWeights = np.full((vehicleNumTot), .5) # Initialize the sampling weights to 0.5 - similarly, can be switched to a dict
    vehicles[i].otherPriorities = np.zeros((vehicleNumTot)) # Initialize list of priorities
    vehicles[i].sampling = vehicles[i].nearbyOBUs.tolist() # Initialize list of OBUs we are sampling
    vehicles[i].id = i # Save vehicles id

historicLoss = {}
selections = {}
results = {}

for vehicle in vehicles:
    if vehicle.id in selections:
        selections[vehicle.id].append([int(i) for i in (vehicle.updateSelected())]) # Update selected vehicles for next training
    else:
        selections[vehicle.id] = [[int(i) for i in (vehicle.updateSelected())]]

for epoch in range(totEpochs):
    print(f"Starting Epoch {epoch} now")
    updatePriorities(vehicles)
    for vehicle in vehicles:
        vehicle.updateSavedStates() # Save current model as model to send to others, so that they are getting the latest after each loop
    for vehicle in vehicles:
         # Model aggregation - Sum weights of all participating models weighted by their priority
        sum = None
        for i in vehicle.sampling:
            if not sum:
                state = vehicles[i].getSavedState()
                sum = dict((n, state.get(n, 0)*vehicle.otherPriorities[i]) for n in state) # Multiplying weights and priority
            else:
                state = vehicles[i].getSavedState()
                add = dict((n, state.get(n, 0)*vehicle.otherPriorities[i]) for n in state) # Multiply weights and priority
                sum = dict( (n, add.get(n, 0)+sum.get(n, 0)) for n in sum) # Add this models weights to the sum
        state = vehicle.getSavedState()
        add = dict((n, state.get(n, 0)*vehicle.priority) for n in state)
        if sum:
            sum = dict((n, add.get(n, 0)+sum.get(n, 0)) for n in sum) # Add this vehicles model to the aggregation
        else:
            sum = add
        vehicle.setState(sum)
        loss = vehicle.step(stepsPerEpoch) # Run training step to progress model
        if vehicle.id in historicLoss:
            historicLoss[vehicle.id].append(loss.item())
        else:
            historicLoss[vehicle.id] = [loss.item()]
        phi(vehicle) # Update theta and confusion matrix
        if vehicle.id in selections:
            selections[vehicle.id].append([int(i) for i in (vehicle.updateSelected())]) # Update selected vehicles for next training
        else:
            selections[vehicle.id] = [[int(i) for i in (vehicle.updateSelected())]]

sampleSizes = {}
for id in selections:
    for epoch in selections[id]:
        if id in sampleSizes:
            sampleSizes[id].append(len(epoch))
        else:
            sampleSizes[id] = [len(epoch)] # Printing number of vehicles sampled by each vehicle each iteration
print(sampleSizes, file=open(f'out/{path}SampleSizes.txt', 'w'))
print(historicLoss, file=open(f'out/{path}HistoricLoss.txt', 'w'))
for vehicle in vehicles:
    results[vehicle.id] = vehicle.test(testDataIn, testDataOut)
print(selections, file=open(f'out/{path}SelectedVehicles.txt', 'w'))
avgF1 = 0
cnt = 0
for result in results: # Loop through results
    print(results[result].split())
    if not vehicles[result].isEvil(): # If the vehicle is evil/misbehaving, we don't include that in our average F1 score.
        if results[result] != 'Model could not complete tests, found 0 of misbehaviour.':
            avgF1 += float(results[result].split()[-1])
            cnt += 1
        else:
            avgF1 += 0
            cnt += 1
avgF1 = avgF1/cnt
results['Final'] = f"Average F1 over dataset: {avgF1}"
print(results, file=open(f'out/{path}DeFedResults.txt', 'w'))
evils = {}
for vehicle in vehicles: # Create list of evil/bad vehicles
    evils[vehicle.id] = f"This vehicle did not contribute a malitious model." if not vehicle.isEvil() else f"This vehicle contributed a false model."
print(evils, file=open(f'out/{path}VehicleStatus.txt','w'))
print(selectionWeights, file = open(f'out/{path}SelectionWeights.txt', 'w')) # Print out the chances of selecting each vehicle

'''
Results:
- 0 Starting weight: Accuracy: 0.2970971516501835, Precision: 0.2970971516501835, Recall: 1.0, F1 Score: 0.4580954499394479
-.5 Starting weight: Accuracy: 0.2970971516501835, Precision: 0.2970971516501835, Recall: 1.0, F1 Score: 0.4580954499394479
'''

'''
Pseudocode:
nodes = list of nodes in simulation
    every node has: N = list of nodes in network; C = list of confidence values; theta = list of weights for each vehicles contribution; D = local dataset; d = outdegree; w = local weights for model
N = list of networks (each node i has a list of attached nodes) * At some point make this vary per iteration?
for each node in simulation in parrallel:
    Initialize list of nodes to be sampled - S
        *** How do we do this ***
    for each epoch:
        collect weights from each node in N
        new w = sum(sample weight * model weight) for model in sampled models - aggregate w
        w = w - (learning rate) * (Loss function (model)) - Local optimizing (training step)
        C_i, theta_i = phi(c, model) - update c and theta
        Update S? - WeightedSample(W, theta)
        Send local data to other models - maybe not necessary depending on how I do this
        wait for the rest of peers to finish this loop

        c is based on the training loss, ie. c_i_j increases if the training loss of w_i decreases (from their paper, they suggest coming up with your own)
        theta = softmax(cRELU(c))
        cRELU = {x; x<= 0, ax; x >0} They suggest a = 0.2 from their findings

        phi(c, w_i):
            m is binary matrix where its j-th entry is 1 iff j is in the set of used models (S) - m has all values on different columns
            p is the set of aggregation weights for all nodes in the network - P has all values on different rows
            if w_i is bad:
                w = w_backup - don't allow bad training to mess up model
                do training step on w
                loss_trust = inf.
            else:
                if loss_curr is lowest:
                    w_backup = w - create backup
                loss_trust = loss_curr - loss_last
                loss_last = loss_curr
            c = c_prev - (m composite multipied by p) * loss_trust - update c
            theta = softmax(cRELU(c)) - Update sample weights
            Update S with subset of N, random sampling using weights to randomize selection size as well as selected values

            
'''

Seed set to 1000
Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/opt/anaconda3/envs/Kettering/lib/python3.13/site-packages/pytorch_lightning/trainer/setup.py:177: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.
Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
HPU available: F

Starting Epoch 0 now
vehicle 0 priority: 0.10173169034265363, total priority of subgroup: 1.0
vehicle 1 priority: 0.006932409012131715, total priority of subgroup: 1.0
vehicle 2 priority: 0.11768241179910337, total priority of subgroup: 1.0
vehicle 3 priority: 0.074585416170753, total priority of subgroup: 1.0
vehicle 4 priority: 0.10787185567489656, total priority of subgroup: 1.0
vehicle 5 priority: 0.011144367318346813, total priority of subgroup: 1.0
vehicle 6 priority: 0.04780267314050419, total priority of subgroup: 1.0
vehicle 7 priority: 0.3357664233576642, total priority of subgroup: 1.0
vehicle 8 priority: 0.036072144288577156, total priority of subgroup: 1.0
vehicle 9 priority: 0.50323978670519, total priority of subgroup: 1.0
vehicle 10 priority: 0.02701912392537575, total priority of subgroup: 1.0
vehicle 11 priority: 0.49015500628403846, total priority of subgroup: 1.0
vehicle 12 priority: 0.24169446008576256, total priority of subgroup: 1.0
vehicle 13 priority: 0.2007168


  | Name     | Type             | Params | Mode 
------------------------------------------------------
0 | model    | Modena           | 1.6 K  | train
1 | lossFunc | CrossEntropyLoss | 0      | train
------------------------------------------------------
1.3 K     Trainable params
264       Non-trainable params
1.6 K     Total params
0.006     Total estimated model params size (MB)
27        Modules in train mode
0         Modules in eval mode
/opt/anaconda3/envs/Kettering/lib/python3.13/site-packages/pytorch_lightning/loops/fit_loop.py:310: The number of training batches (2) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Epoch 4: 100%|██████████| 2/2 [00:00<00:00, 127.76it/s, v_num=3815, trainLoss=2.530]

`Trainer.fit` stopped: `max_epochs=5` reached.


...


  | Name     | Type             | Params | Mode 
------------------------------------------------------
0 | model    | Modena           | 1.6 K  | train
1 | lossFunc | CrossEntropyLoss | 0      | train
------------------------------------------------------
1.3 K     Trainable params
264       Non-trainable params
1.6 K     Total params
0.006     Total estimated model params size (MB)
27        Modules in train mode
0         Modules in eval mode


[-3144.52806582     4.             4.             4.
  -229.75731175     4.             4.             4.
     4.             4.             4.             4.
     4.          -956.67955222     4.             4.
  -125.60212798     4.             4.             4.
     4.             4.           -31.47362689   -89.10402454
     4.          -271.88126939  -204.40487751     4.
     4.             4.             4.             4.
  -661.07119391     4.          -562.00356275     4.
     4.             4.             4.         -1394.72288101
     4.         -1454.27344501     4.             4.
  -282.08294926     4.             4.             4.
     4.          -424.48786714]
Epoch 149: 100%|██████████| 2/2 [00:00<00:00, 103.17it/s, v_num=3864, trainLoss=0.365]

`Trainer.fit` stopped: `max_epochs=150` reached.


Epoch 149: 100%|██████████| 2/2 [00:00<00:00, 87.14it/s, v_num=3864, trainLoss=0.365] 
[4.         4.02059269 4.28442907 4.         4.         4.01197529
 4.         4.         4.         4.         4.04514074 4.55926561
 4.         4.13437557 4.1667757  4.         4.         4.
 4.         4.         4.         4.         4.         4.03129339
 4.         4.         4.09596777 4.         4.         4.
 4.         4.         4.         4.         4.35474062 4.
 4.         4.         4.         4.         4.         4.
 4.         4.         4.         4.         4.         4.
 4.         4.        ]
torch.Size([124774, 10, 20])
Model could not complete tests.
torch.Size([124774, 10, 20])
Model could not complete tests.
torch.Size([124774, 10, 20])
Model could not complete tests.
torch.Size([124774, 10, 20])
Model could not complete tests.
torch.Size([124774, 10, 20])
Model could not complete tests.
torch.Size([124774, 10, 20])
Model could not complete tests.
torch.Size([124774, 10, 20]

"\nPseudocode:\nnodes = list of nodes in simulation\n    every node has: N = list of nodes in network; C = list of confidence values; theta = list of weights for each vehicles contribution; D = local dataset; d = outdegree; w = local weights for model\nN = list of networks (each node i has a list of attached nodes) * At some point make this vary per iteration?\nfor each node in simulation in parrallel:\n    Initialize list of nodes to be sampled - S\n        *** How do we do this ***\n    for each epoch:\n        collect weights from each node in N\n        new w = sum(sample weight * model weight) for model in sampled models - aggregate w\n        w = w - (learning rate) * (Loss function (model)) - Local optimizing (training step)\n        C_i, theta_i = phi(c, model) - update c and theta\n        Update S? - WeightedSample(W, theta)\n        Send local data to other models - maybe not necessary depending on how I do this\n        wait for the rest of peers to finish this loop\n\n    

In [ ]:
# TESTING
d = {'a': 1, 'b': 2, 'c': 3}
e = ['a','b']
print([d[x] for x in e])

[1, 2]


In [ ]:
# TESTING
print(weights)
i=0
for w in weights:
    i+= w

print(i)

[0.10002457 0.09977863 0.10002469 0.10002465 0.10002414 0.10002459
 0.10002469 0.10002469 0.10002468 0.10002469]
1.0000000000000002


In [ ]:
# TESTING
cumModel = hiddens[0][2]
iter = 1
for model in hiddens[1:]:
    if model[1] != -1:
        if iter == 0:
            cumModel = model[2]
        else:
            cumModel = cumModel + model[2]
        iter += 1
    else:
        print(model[0])
        print(cumModel/iter)
        print(model[2])
        print(cumModel/iter - model[2])
        iter = 0

0
tensor([[-0.9981, -1.0000,  1.0000, -0.8293, -1.0000, -1.0000,  1.0000, -1.0000,
          0.6075,  0.0442,  0.7163,  0.6767,  0.7933,  0.7151,  0.5629,  0.9533,
         -0.8753,  0.7856, -0.5999, -0.2247],
        [-0.9155, -1.0000,  1.0000, -0.7490, -1.0000, -1.0000,  1.0000, -1.0000,
          0.5863,  0.0659,  0.7266,  0.6708,  0.7904,  0.7140,  0.5576,  0.9531,
         -0.8754,  0.7839, -0.6045, -0.2218],
        [-0.9489, -1.0000,  1.0000, -0.8079, -1.0000, -1.0000,  1.0000, -1.0000,
          0.6044,  0.0083,  0.7259,  0.6747,  0.8049,  0.7217,  0.5656,  0.9527,
         -0.8770,  0.7812, -0.5921, -0.2208],
        [-0.9244, -1.0000,  1.0000, -0.8202, -1.0000, -1.0000,  1.0000, -1.0000,
          0.5903,  0.0450,  0.7244,  0.6746,  0.7943,  0.7130,  0.5605,  0.9540,
         -0.8758,  0.7817, -0.6040, -0.2236],
        [-0.8204, -1.0000,  1.0000, -0.8306, -1.0000, -1.0000,  1.0000, -1.0000,
          0.5760,  0.0688,  0.7257,  0.6738,  0.7918,  0.7142,  0.5560,  0.9537,
    

In [ ]:
# TESTING
list = [0,1,2]
print(list[0:])

[0, 1, 2]


In [ ]:
# INDIVIDUAL OBU TEST SETUP
testOBU = OBU(
    inputSize = 7,  # Number of features per BSM
    units = 20, # Number of hidden cells
    motors = 8, # Number of motor neurons
    outputs = 20, # Number of possible labels
    epochs = 50,
    lr = 0.01,
    gpu = True
)

Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [ ]:
# TRAIN INDIVIDUAL OBU
testOBU.fit(testingDataLoader)

/opt/anaconda3/envs/Kettering/lib/python3.13/site-packages/pytorch_lightning/trainer/configuration_validator.py:70: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.

  | Name     | Type             | Params | Mode 
------------------------------------------------------
0 | model    | Modena           | 1.6 K  | train
1 | lossFunc | CrossEntropyLoss | 0      | train
------------------------------------------------------
1.3 K     Trainable params
264       Non-trainable params
1.6 K     Total params
0.006     Total estimated model params size (MB)
27        Modules in train mode
0         Modules in eval mode


Epoch 49: 100%|██████████| 78/78 [00:04<00:00, 19.00it/s, v_num=634, trainLoss=2.520]

`Trainer.fit` stopped: `max_epochs=50` reached.


Epoch 49: 100%|██████████| 78/78 [00:04<00:00, 18.91it/s, v_num=634, trainLoss=2.520]


tensor(2.5195, device='mps:0', grad_fn=<NllLoss2DBackward0>)

In [ ]:
testOBU.fit(testingDataLoader)

/opt/anaconda3/envs/Kettering/lib/python3.13/site-packages/pytorch_lightning/trainer/configuration_validator.py:70: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.

  | Name     | Type             | Params | Mode 
------------------------------------------------------
0 | model    | Modena           | 1.6 K  | train
1 | lossFunc | CrossEntropyLoss | 0      | train
------------------------------------------------------
1.3 K     Trainable params
264       Non-trainable params
1.6 K     Total params
0.006     Total estimated model params size (MB)
27        Modules in train mode
0         Modules in eval mode
/opt/anaconda3/envs/Kettering/lib/python3.13/site-packages/torch/utils/data/dataloader.py:626: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 10 (`cpuset` is not taken into account), which is smaller than what this DataLoader is going to create. Please be aware that exces

Epoch 2:   9%|▉         | 7/78 [00:00<00:03, 18.12it/s, v_num=595, trainLoss=1.800] 


Detected KeyboardInterrupt, attempting graceful shutdown ...
libc++abi: terminating due to uncaught exception of type std::__1::system_error: Broken pipe
libc++abi: terminating due to uncaught exception of type std::__1::system_error: Broken pipe
libc++abi: terminating due to uncaught exception of type std::__1::system_error: Broken pipe
libc++abi: terminating due to uncaught exception of type std::__1::system_error: Broken pipe
libc++abi: terminating due to uncaught exception of type std::__1::system_error: Broken pipe
libc++abi: libc++abi: terminating due to uncaught exception of type std::__1::system_error: Broken pipe
terminating due to uncaught exception of type std::__1::system_error: Broken pipe
libc++abi: terminating due to uncaught exception of type std::__1::system_error: Broken pipe
libc++abi: terminating due to uncaught exception of type std::__1::system_error: Broken pipe
libc++abi: terminating due to uncaught exception of type std::__1::system_error: Broken pipe
libc++ab

: 

In [ ]:
testOBU.test(testDataIn, testDataOut)

torch.Size([124774, 10, 20])
0.2970971516501835
1.0
Model got 370700/1247740 right.
Accuracy: 0.2970971516501835, Precision: 0.2970971516501835, Recall: 1.0, F1 Score: 0.4580954499394479
877040, 70.29028483498165% Zeroes, 370700 Non Zero entries.


'Model got 370700/1247740 right. Accuracy: 0.2970971516501835, Precision: 0.2970971516501835, Recall: 1.0, F1 Score: 0.4580954499394479'

In [ ]:
# TESTING
save = OBU(testOBU)

Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [52]:
# RUN TEST ON SMALL DATASET INDIVIDUAL MODEL
_, _, perc, _ = testOBU.test(inTest, outTest)

torch.Size([12480, 10, 20])
Model got 6232/124800 right. Accuracy of 4.993589743589744%
69.88782051282051% Zeroes.


In [ ]:
# RUN TEST ON LARGE DATASET INDIVIDUAL MODEL
_, _, perc, _ = testOBU.test(testDataIn, testDataOut)

torch.Size([196990, 10, 20])
Model got 1159988/1969900 right. Accuracy of 58.88562871211737%
40.353165135286055% Zeroes.


In [ ]:
# TESTING COPY
_, _, perc, _ = save.test(inTest, outTest)

torch.Size([19700, 10, 20])
Model got 116065/197000 right. Accuracy of 58.91624365482233%
41.08375634517766% Zeroes.


In [ ]:
# TESTING
print(testOBU.getHidden() - save.getHidden())

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0

In [ ]:
# TESTING
print(testOBU.model.fF.weight - save.model.fF.weight)

tensor([[0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.]], grad_fn=<SubBackward0>)
